<h1 align="center">Functions Part 2 - An Application Approach</h1>

From here on out we will concentrate primarily on build a toy application which will create 'pretty' pictures of a non-linear equation fun-storm called Strange Attractors. In our process of developing a full fledged Qt based GUI'ed toy application we will learn more about functions in addition we will look and several other Python packages that will prove both helpful now and in your future if you continue on down the Python pathway. This whole process will mimic a real world development (albeit as presented - a hacking method) complete with problem solving to get around *'issues'* we will intentional and unintentionally run into.

# Strange Attractors

From here on out we are going to work with an application based approach for our examples and slowly build a full fledge, GUIed application which will display a handful of different strange attractors.

One of the better, one sentence definition of a strange attractor is: "A stable, non-periodic mathematical behavior exhibited by some dynamically chaotic systems that can be represented as a non-repeating pattern in the system's phase space." *The* classic example is the first to be described by the atmospheric physicist, Edward Lorenz and his *Lorenz attractor*. These attractors can exists in 2, 3 or even much much higher dimensions. 

While there are important scientific applications for attractors we will just treat them as pretty pictures!

As mentioned above these equations, as far we we will concern ourselves with a either 2D or 3D mathematical equations and all of the have one thing in common, they are calculated iteratively. Meaning X[i+1] = X[i]+some fxn.<br>
All attractors have a few things in common.<br>
- They are iterative so we run a *for* or *while* loop on equations.
- The equations are non-linear equations with final equation for each dimension.
- The non-linear equations have a set number of parameters.

The number of parameters used range from a few and the largest I have worked with had 167!!! And yes you can go MUCH larger you just have to manhandle all those parameters and it gets sticky fast! We will confine ourselves to attractors with no more then 7 parameters.


**Example: The Clifford Attractor**
For most of the work and examples we will work with centers on what is known as the *Clifford attractor*. This attractor is a 2D attractor taking 4 parameters a, b, c,and d defined by the following set of equations.

\begin{align*}
x[i+1] = sin(a * y[i]) + c * cos(a * x[i]) \\
y[i+1] = sin(b * x[i]) + d * cos(b * y[i])
\end{align*}


To see some other of the huge zoo of attractors out there I suggest starting here: http://www.chaoscope.org/doc/attractors.htm<br>
and here: http://paulbourke.net/fractals/

The attractors and their example parameter values we will be using came from these resources (ages ago, but these pages are ancient as far as the WebWorld goes.)


### Exercise:
- Create a function named Clifford
- This function takes 5 parameters x,y,a,b,c,d
- The function creates 2 variables Xn and Yn these variables hold the respective x[i+1] and y[i+1] equations described above.
- The function returns both Xn and Yn

Next we need to create the iterative code to create the *trajectory* of the particles for each iterative time step.

We will create the Trajectory function to take a function name as a variable (fxn) holding a string which is the name of the function we want to call! The number of iterations (n) and the initial X and Y position and the 4 Clifford parameters. We then setup a arrays for each dimensional equation and then iterate across all n iterations sending the current x and y values to the Clifford equation function and receiving the updated position data for x[i+1] and y[i+i].<br>

Finally return a special dataframe structure holding all the data points generated. We will discuss the importance of this dataframe structure more shortly.

The new Trajectory function looks like this:

In [ ]:
def trajectory(fxn, n, x0, y0, a, b, c, d):
    """ Iterates over all desired iterations calculating the new position of the particle 
        at each iteration based on the strange attractor asked for.
        Right now we only have the Clifford Attractor to work with.
    """
    # Initialize x and y particle variables to the size of n (number of iterations)
    x, y = np.zeros(n), np.zeros(n)
    
    #Set the inital postions to the configurations initial positions
    x[0], y[0] = x0, y0
    
    #Iterate over all of n calculating new points using the desired strange attractor equation.
    #Note the cool trick of calling the function via a string holding the name in a variable (fxn)
    for i in np.arange(n-1):
        x[i+1], y[i+1], = fxn(x[i], y[i], a, b, c, d)

    #Create a new dataframe with all of the particle information (x,y vars)
    return pd.DataFrame(dict(x=x, y=y))

In order to appreciate the real beauty of most attractors you need to run an obscenely large number of iterations noting that there will never be any two points in exactly the same gridded space (thats part of their definition).

This means that we will need to run iterations from 100 thousand to >1 billion. But running the calculations is the easy half of the chore. We also need to be able to render a billion or more points of data. OUCH, now thats a MAJOR issue!!!

(Note this is worse then it sounds since thats 1 billion data points per dimension. So if you are working with a 3 dimensional attractor you will have 3 billion data points!!! Thank heavens for cheap memory, I just hope you have some :) )

Oh wait we're working with Python, crisis averted thanks to a package called ***Datashader***.

## Datashader - Plotting large to extremely large data in near RealTime

Using their definition:<br>
*"Datashader is a graphics pipeline system for creating meaningful representations of large datasets quickly and flexibly. Datashader breaks the creation of images into a series of explicit steps that allow computations to be done on intermediate representations. This approach allows accurate and effective visualizations to be produced automatically without trial-and-error parameter tuning, and also makes it simple for data scientists to focus on particular data and relationships of interest in a principled way."*

If you have a huge amount of data you want to plot and it can be projected onto a 2D grid then you really want to dive deep into Datashader. Datashader actually is part of a pseudo suite of Python plotting routines consisting of Bokeh, Holoviews, Geoviews and, of course, Datashader. I have in the past and am planning on doing so again this fall, taught a series of workshops that cover this suite. If you are interested in learning more follow the CRDDS (CU's Center for Data Research and Digital Scholarship) 'events' schedule. https://www.colorado.edu/crdds/

For more information on Datashader please visit: http://datashader.org/<br>
Note, I just found they have a different tutorial on strange attractor rendering there as well now too. It could be of interest as they use a yml file parser and a slightly different way of passing/parsing the attractor parameters. Plus they have multiple parameter values for the same attractor so you can see how to deal with that as well. 



### Datashader for Attractors

First note that Datashader is only a 2D image plotter! This will slightly effect us later when we deal with 3D attractors but it's actually a minor inconvenience.  

One of the most common and basic ways of rendering strange attractors is to use a gridded aggregate method. This means that you save all your data to a grid and then scan cell by cell across the grid. For each cell you add up the total number of times a point fell within the domain of the cell. After you finish scanning the grid you apply a transfer function to the grid to transfer the value of the aggregate to a colormap (***cmap***) where each color corresponds to a unique value in the range of all aggregate values. If you have seen images of strange attractors before, beyond Lorenz attractors, this is probably the coloring method you have seen.

Other common and not so common ways of coloring attractors is a velocity method of how fast the particle in each iteration is moving around basins of attraction. The easiest way is to color by time, such that the first iteration is time=0 and the last iteration is time=tmax and setting the transfer function to be bounded there in. These are actually fairly ugly mappings but you can see how fast particles are whipping around (at least sometimes, other times things get muddy real quick).

We will use the aggregation method as its both a 'pretty' solution and its a basic method Datashader makes available to use.

**A personal quick aside on a potential value coloring and the study of attractors for real world benefits**
Theres a lot of scientific meaning using this method or funkier yet, coloring by acceleration. This has value if looking at the topology of the limit cycles surrounding the basins of attraction. if you are doing some deep analysis on the limit cycles I found you can actually perform Floquet analysis on you system of equations and then color by the shift in the real coefficients. Why? It helps if you are 'trying' to define the boundaries of the limit cycle. I say try as in the end its really just a probability cloud you are creating so the coloring really becomes a function of the probability you are near an edge of the limit cycle. OK again why? All locomotion, human, dog, bird, bug, bacterial flagella is embedded into a nonlinear limit cycle that looks exactly like strange attractors. So, if like something I used to work on, you were trying to understand the limit cycle for human gait (eg walking, running, ect) in hopes of designing '6 Million Dollar Man' or RoboCop type legs for the real world, then you need to understand the boundaries of the limit cycle inorder to maintain balance and motion.  



### Using Datashader

Using Datashaders built in aggregation method there is basically 3 things you need to do to plot you data.
- Create a *Canvas*, which defines the grid you will plot onto. You need to specify the plots width and height, which naturally becomes the resolution of the plot. This is akin to Matplotlib's Figure creation.
- Assign the point data to the canvas (more on this in a second). 
- Shade(color) the points with a colormap.

The second step is the confusing step, at least the first time through. If remember a few minutes ago we created a special 'dataframe' using Pandas (we will cover Pandas a bit more in a few minutes). This dataframe contains a simple dictionary with the key being one of the axis (X,Y, and potentially Z) and the value being the data points for that axis. <br>
Datashaders Canvas.points() function requires the data being in a dataframe and also takes the keys of the dictionary to define the axis. Important to remember Datashader can only handle 2D plots but we can create a dataframe with as many spatial dimensions we want. You just have to confine your self to only plotting two of those dimensions at any given time (e.g. XY, XZ, YZ, ect...)


So next we create the basic datashader plot function (dsplot).

For purposes this function takes several parameters:<br>
- (fn) The name of attractor function (for later use)
- (n) The number of iterations we swish to run
- (cmap) The name of the color map we want. We will use a subset of colorcet's colormaps
- The initial positions for x, y, z (z will be 0 if we are using a 2D equation)
- A parameter for each attractor parameter, in this case 4 for a, b, c, and d

The function first creates the cmap from the colorcet colormap we desire to color with.<br>
We then call our trajectory function to run all our iterations and buld our dataframe to plot with.
Then we create an image (img) using Datashader 
We return the img to be displayed later.

To do all of this you need 2 to 3 Datashader imports

```python
import datashader as ds
from datashader import transfer_functions as tf
from datashader import utils

from colorcet import palette
```

The third datashader import, *utils* has a function in it to save the image to disk and we will look at that later.<br>
The last import imports all of colorcets palettes.

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from datashader import utils

def dsplot(fn, n, cmap, x0, y0, a, b, c, d):
    """Return a Datashader image by collecting `n` trajectory points for the given attractor `fn`"""
    cmap = palette[cmap][::-1]
    df  = trajectory(fn, n, x0, y0, a, b, c, d)
    
    cvs = ds.Canvas(plot_width = 400, plot_height = 400)
    agg = cvs.points(df, 'x', 'y')
    img = tf.shade(agg, cmap=cmap)
    
    return img

We now have the everything we need to create our first strange attractor script.

### Exercise: ###

Copy the above function blocks (Clifford(), trajectory(), and dsplot()) into the cell below, placing them above the lines already present.

Those additional lines define the number of iterations we will run (*n*), the name of the colormap we want to use (*bgy* - for blue-green-yellow), a call to our dsplot()function with the preceding values as well as the initial particle positions and the values for the Clifford attractor parameter. The, being in a Jupyter Notebook we display the final image. 

Once done go ahead and run the cell to create our first attractor.

## TIMING
Python provides a module, 'time', to provide timing functionality. We have seen it takes a while to run 1 million iterations but lets find out just how long, long is.<br>
Once you import the time module you just need to call a 'time' function from the 'time' class this. this will return the current system time.


**Example:**

```python
import time

start = time.time()
#do something
end = time.time()

runtime = end - start
```


We will use this method to create a start and end time around the call to dsplot to find the total runtime of our script.

### Exercise:
- Copy the above code to the cell below
- Import time
- Add a timing code around the img = dsplot() line

Well now we know that it takes a few seconds to render just 1 million iterations. We want to be able to run 1 billion iterations. The problem here, or benefit in some ways, is that you have to remember we are working with a purely iterative and thus linear operation. Thus we can predict if we increase the iterations by one magnitude it will likewise take a magnitude longer to run. 

So what to do, what to do?!?!

We could work towards optimizing the code except there is not much we can do and it want really buy us much. We can try creating a cleaner object to encapsulate our code. We will actually do this in a few minutes, but that won't provide a major performance increase but it will give us vastly more power to what we can do with our code.

Orrrrrr, we can jit it!!!

## Numba and  JIT

So heres the major flaw with Python, yes it has a major kryptonite weakness and thats that it is NOT a compiled language. Skipping all real and gory details it uses a Just In Time Compilation (JIT) process to translate your code to something the computer can execute. This gets exponentially more complicated from here leading all the way to you can indeed build a Python compiler, in fact you can use Python to write a Python compiler but there is limitations on what you can do.

However this is note what I mean by *'lets jit it'*.

There is another module called ***Numba*** which is indeed a JIT compiler (to an extent). It sadly has some major limitations but for us it has some major power. Well up until we run into some of those limitations.

For us, all we need to do it import numba and its jit compiler and then run a decorator call (think pragma) before the function we want to pre-compile.

**Example:**

```python
from numba import jit

@jit
def foo():
    do something heavily numpy/mathematically based and enjoy a major speedup.
```

More information on Numba can be found at: http://numba.pydata.org/

## Decorators

In Python decorators are a powerful tool which allows you to take a function and literally *'wrap'* it inside another object (basically a function or class).

Decorators are very powerful and useful tool in Python since it allows programmers to modify the behavior of function or class. Decorators allow us to wrap another function in order to extend the behavior of wrapped function, without permanently modifying it.

In Decorators, functions are taken as the argument into another function and then called inside the wrapper function. Another way to look at it is that there is some function out there which has a name used in the decorator and it takes a function as a parameter.

Decorators are prefaced with an *'@'* symbol followed by the decorator name. All of this is in the line right above the function you want the decorator to act on.

So lets now apply that to out Attractor code.

### Exercise:
- Copy the above timing code to the cell below
- Import jit
- Add a jit decorator to the Clifford and Trajectory functions.
- Run the code and figure out the approximate % speedup jit provides.

## Pandas - Loading a Configuration File

Pandas is a Python swiss army knife for working with data and data files. Its not the perfect end-all be-all, for instance no HDF5 support and while it can actually handle netCDF you are still much better off using the separate ***netCDF4*** package.

Pandas's real advantage is not that it reads and writes many commonly generic file formats but that when you load them in they are loaded into a special ***dataframe*** a special tabular data structure which allows all kinds of powerful tools for data wrangling and mangling.

You have already used Pandas to create a special dataframe on the fly which is required for Datashader to create its renders.

Like Numpy its tradition to load Pandas as an instance named ***'pd'***

We will use Pandas to read a special ***.atr*** which has a series of configurations for about a dozen different strange attractors. This .atr file is really just an .cvs file which was renamed to be an new .atr file? Why? Because you can and Pandas as you will see could care less about the rose, err I mean name.

The function Pandas uses to open a csv file is read_cvs() which is a member of the Pandas class and takes a mandatory parameter of the name of the file you want to open.

**Example:**

```python
import pandas as pd

fileName = 'foo.cvs'
df = pd.read_cvs(filName)
```

### Exercise:
- Use the above example to open the ***'attractors.atr'*** file
- Use Pandas ***head()*** function to display the header and first 5 rows of the new dataframe(df)

It's easy to read dataframe columns especially if you have a header line. The dataframe is can be treated as what it is a specialized 2D Numpy-esque (yes you read that right Numpy) array. In fact Pandas utilizes Numpy for a great many of its vast array (no pun intended just to stupid to think of another word) of tools.

So to read the Attractor column you call a Pandas location function called ***.loc()***. However you treat it as an array like structure so instead of ()'s you use []'s where you treat the array as [row, column] based. The cool thing here is you can use the dataframes column header name for the column. Else you just use normal Numpy slicing.

So to read the 'Attractor' column you would just do;

In [ ]:
df.loc[: , 'Attractor']

For our upcoming purpose we need row data and finding the data for a particular row is slightly more complicated. Before you call the location (loc[]) function you need to define the index column you will use to find the row, or rows, you seek.

You set the index using the set_index function. In our case we use to parameters, the column name to index and an 'inplace = True' parameter to tell Pandas  to change the current dataframe and that we are not creating a new one.



In [ ]:
df.set_index('Attractor', inplace=True) 
fxn = 'Clifford'
a = df.loc[fxn]

a

Please note a few key points which you can see and something you can't with this example.
- We no longer needed to do any slicing to find the data for our desired row (this time!)
- All the data that use to be in a row form is now in column form.
- This applies not only to the data but the header as well.
- What this example does not show is that it will actually find ALL instances of a Attractor row with values of 'Clifford' or whatever we search for. To deal with them you can just slice away.

## Creating the File Read and Getting Attractor Config Functions

Now that we have a file with various different attractors and a set of associated initial positions and parameter values we need to refactor our code to take that information and then send a specified (we will work mainly with *Clifford* for now) set of attractor data and send it off to be calculated and render.



The first thing we want to do is to create an actual function to open and read our file. We will call our new function *'get_AttractorConfigFile()'*

When you look at the new function below you will notice some additional error handling code wrapped around our simple -read_csv() and set_index() call lines. the purpose here is to make sure the file exists where we tell it to look for it and if it does exist can it actually be read. If not gracefully post a semi-descriptive error message and exit the script.

**Exception Handling**
Normally you would handle this error check using exception handling and I recommend if you are using something other then Pandas or xarray (a pandas like package that is used for large scientific data.) We will not be using it here as Pandas has it built in and it gets a little sticky mixing your exception handling and our own so for Pandas let Pandas do the work for you. 

File handling is not the only time you want to deal with exception handling but alas that, sadly, is a different lesson for a different day. 


So our get_AttractorConfigFile() function looks like this:

In [ ]:
def get_AttractorConfigFile(fName):
    """Open the Attractors configuration file"""
    if os.path.isfile(fName) and os.access(fName, os.R_OK):
        df = pd.read_csv(fName)
        df.set_index('Attractor', inplace=True)   
    else:
        print("ERROR: The file is either missing or it's not readable")
        sys.exit(1)

Next up we need to parse the data values for a desired attractor and return the initial positions (coords)and the attractor parameter values (params). To do this we make use of Pandas's location function (*loc()*). 

We will also make use of a sister location function called *iloc()* which locates data in a dataframe based on an index(in this case position) value. Looking at the data for the Clifford attractor we see:<br>
***Clifford  bgy  0.0  0.0  0.0  -1.900000  -1.900000  -1.900000  -1.000000  NaN  NaN  NaN***

After indexing based on Attractor the 'Clifford' column is an index column and the cmap column is our new 0th column. To get the x, y, and z coordinates we need the respective data from indices 1, 2, and 3. All other parameters fall into indices 4 - 10.

So to get the attractor configuration data we create the following *getAttractor()* function.


In [ ]:
def getAttractor(fxn):
        """Get the configuration for the desired attractor"""
        a = df.loc[fxn]

        #Get the initial positions (x,y,z) data and store in coords (paritcle coordinates)
        coords = [a.iloc[1], a.iloc[2], a.iloc[3]]

        #Get the rest of the attractors parameters
        params = np.array([])
        for v in range(4, np.size(a)):
            params = np.append(params, a.iloc[v])

        #Return the inital coodrinates and attractor parameters
        return coords, params

## Next Up

We are going to call this section good for now regardless of the fact that we are half way between our original attractor code and something much sexier. But to get to the sexy part we need to move onto classes.

**Some xarray soapbox preaching **

In fact xarray is so much like Pandas that it is dangerous for a sane man to use. Say what? It is designed to be a high performance large data Panads substitute down to using the exact same function names, parameter spaces and returns. BUT it is not as mature as Pandas YET! Meaning that when working with xarray and you will tend to forget you are indeed working with it and try to do something you know Pandas can do and then waste a half day or a lot more trying to figure what the heck is going on with your Pandas, err xarray code. I've done it a few times and I have seen many others do it repeatedly. 

Am I saying don't use it? Heck no, the thing is a wonderous beast and in fact the preferred method to working with Datashader is with the use of xarray and another high performance package called Dask. In the future there will be a whole module to this training devoted to incorporating it into our project. Dask will not help with the base attractor computations but there are other things we can do with the data that will make dealing with a billion data points a breeze!

For information of xarray please visit: http://xarray.pydata.org/en/stable/why-xarray.html<br>
For information of Dask please visit: https://docs.dask.org/en/latest/